In [217]:
import pandas as pd

In [218]:
SUFFIX = "2021-2023_20250526"
PREFIX = "../data/"

TRADES_PATH = f"{PREFIX}trades/trades.csv"
VOTES_PATH = f"{PREFIX}votes/votes_by_member_{SUFFIX}.csv"
VOTE_SUMMARIES_PATH = f"{PREFIX}votes/votes_summary_{SUFFIX}.csv"

OUTPUT_DIR = "../data/cleaned"

In [219]:
trades_df = pd.read_csv(TRADES_PATH)

In [220]:
print(trades_df["asset_type"].unique())
print(len(trades_df["asset_type"] == "Stock"))
print(trades_df["industry"].unique())
print(trades_df["sector"].unique())
trades_df.head()

[nan 'Stock' 'PDF Disclosed Filing' 'Stock Option' 'Other Securities'
 'Corporate Bond' 'Cryptocurrency']
25616
['Integrated oil Companies' 'Real Estate Investment Trusts'
 'Farming/Seeds/Milling' 'Investment Bankers/Brokers/Service'
 'Finance: Consumer Services' nan 'Semiconductors'
 'Computer Manufacturing' 'Computer Software: Prepackaged Software'
 'Electric Utilities: Central' 'Paper' 'Investment Managers'
 'Major Pharmaceuticals' 'Power Generation' 'EDP Services'
 'Biotechnology: Pharmaceutical Preparations' 'Major Banks'
 'Oil & Gas Production' 'Other Consumer Services' 'Marine Transportation'
 'Biotechnology: Commercial Physical & Biological Resarch'
 'RETAIL: Building Materials' 'Oilfield Services/Equipment'
 'Savings Institutions' 'Other Metals and Minerals'
 'Pharmaceuticals and Biotechnology' 'Professional Services'
 'Movies/Entertainment' 'Real Estate' 'Medical Specialities'
 'Department/Specialty Retail Stores' 'Food Distributors'
 'Office Equipment/Supplies/Services' 'Aut

,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,district,state,ptr_link,cap_gains_over_200_usd,industry,sector,party,asset_type,is_senator,chamber,member
0,10/04/2021,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Integrated oil Companies,Energy,Republican,NaN,False,House,Virginia Foxx
1,10/04/2021,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Integrated oil Companies,Energy,Republican,NaN,False,House,Virginia Foxx
2,10/04/2021,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Real Estate Investment Trusts,Real Estate,Republican,NaN,False,House,Virginia Foxx
3,10/04/2021,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Farming/Seeds/Milling,Consumer Non-Durables,Republican,NaN,False,House,Virginia Foxx
4,10/04/2021,2021-09-17,self,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",CA47,CA,https://disclosures-clerk.house.gov/public_dis...,False,Investment Bankers/Brokers/Service,Finance,Democrat,NaN,False,House,Alan S. Lowenthal


In [221]:
# if ticker exists, set asset type to "Stock"
trades_df["asset_type"] = trades_df["ticker"].apply(
    lambda x: "Stock" if isinstance(x, str) and len(x) > 0 else "Other"
)
# only keep trades with asset_type "Stock"
trades_df = trades_df[trades_df["asset_type"] == "Stock"]

print(trades_df["amount"].unique())

print(trades_df["is_senator"].unique())
trades_df.head()

['$1,001 - $15,000' '$15,001 - $50,000' '$50,001 - $100,000'
 '$100,001 - $250,000' '$1,001 -' '$250,001 - $500,000'
 '$500,001 - $1,000,000' '$5,000,001 - $25,000,000'
 '$1,000,001 - $5,000,000' '$1,000,000 +' '$1,000 - $15,000'
 '$15,000 - $50,000' '$50,000,000 +' '$1,000,000 - $5,000,000'
 '$25,000,001 - $50,000,000']
[False  True]


,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,district,state,ptr_link,cap_gains_over_200_usd,industry,sector,party,asset_type,is_senator,chamber,member
0,10/04/2021,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Integrated oil Companies,Energy,Republican,Stock,False,House,Virginia Foxx
1,10/04/2021,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Integrated oil Companies,Energy,Republican,Stock,False,House,Virginia Foxx
2,10/04/2021,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Real Estate Investment Trusts,Real Estate,Republican,Stock,False,House,Virginia Foxx
3,10/04/2021,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",NC05,NC,https://disclosures-clerk.house.gov/public_dis...,False,Farming/Seeds/Milling,Consumer Non-Durables,Republican,Stock,False,House,Virginia Foxx
4,10/04/2021,2021-09-17,self,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",CA47,CA,https://disclosures-clerk.house.gov/public_dis...,False,Investment Bankers/Brokers/Service,Finance,Democrat,Stock,False,House,Alan S. Lowenthal


In [222]:

# Define mapping from amount range strings to numeric estimates
amount_mapping = {
    "$1,001 - $15,000": 10000,
    "$15,001 - $50,000": 35000,
    "$50,001 - $100,000": 75000,
    "$100,001 - $250,000": 175000,
    "$1,001 -": 1000,
    "$250,001 - $500,000": 375000,
    "$500,001 - $1,000,000": 750000,
    "$5,000,001 - $25,000,000": 15000000,
    "$1,000,001 - $5,000,000": 3000000,
    "$1,000,000 +": 1000000,
    "$1,000 - $15,000": 8000,
    "$15,000 - $50,000": 32500,
    "$50,000,000 +": 50000000,
    "$1,000,000 - $5,000,000": 3000000,
    "$25,000,001 - $50,000,000": 37500000,
}

# Map is_senator to chamber
trades_df["chamber"] = trades_df["is_senator"].apply(
    lambda x: "Senate" if x else "House"
)

# Clean up the "amount" column: strip whitespace
trades_df["amount"] = trades_df["amount"].astype(str).str.strip()

# Optional: normalize formatting if needed (remove commas, extra symbols)
# trades_df["amount"] = trades_df["amount"].str.replace(",", "").str.strip()

# Show unmapped values for debugging
unmapped_amounts = trades_df[~trades_df["amount"].isin(amount_mapping.keys())]["amount"].unique()
print("Unmapped values:\n", unmapped_amounts)

# Map amount to numeric value
trades_df["amount"] = trades_df["amount"].map(amount_mapping)

# Keep only the relevant columns
trades_df = trades_df[
    [
        "transaction_date",
        "ticker",
        "asset_description",
        "type",
        "amount",
        "state",
        "ptr_link",
        "industry",
        "sector",
        "party",
        "chamber",
        "member",
    ]
]

# Rename columns to match schema
new_trades_df = trades_df.rename(
    columns={
        "transaction_date": "transaction_date",
        "ticker": "ticker",
        "asset_description": "asset_description",
        "type": "type",
        "amount": "amount",
        "state": "state",
        "ptr_link": "ptr_link",
        "industry": "industry",
        "sector": "sector",
        "party": "party",
        "chamber": "chamber",
        "member": "member",
    }
)

# Show result
new_trades_df.head()

Unmapped values:
 []


,transaction_date,ticker,asset_description,type,amount,state,ptr_link,industry,sector,party,chamber,member
0,2021-09-27,BP,BP plc,purchase,10000,NC,https://disclosures-clerk.house.gov/public_dis...,Integrated oil Companies,Energy,Republican,House,Virginia Foxx
1,2021-09-13,XOM,Exxon Mobil Corporation,purchase,10000,NC,https://disclosures-clerk.house.gov/public_dis...,Integrated oil Companies,Energy,Republican,House,Virginia Foxx
2,2021-09-10,ILPT,Industrial Logistics Properties Trust - Common...,purchase,35000,NC,https://disclosures-clerk.house.gov/public_dis...,Real Estate Investment Trusts,Real Estate,Republican,House,Virginia Foxx
3,2021-09-28,PM,Phillip Morris International Inc,purchase,35000,NC,https://disclosures-clerk.house.gov/public_dis...,Farming/Seeds/Milling,Consumer Non-Durables,Republican,House,Virginia Foxx
4,2021-09-17,BLK,BlackRock Inc,sale_partial,10000,CA,https://disclosures-clerk.house.gov/public_dis...,Investment Bankers/Brokers/Service,Finance,Democrat,House,Alan S. Lowenthal


In [223]:
print(trades_df["amount"].unique())

# mask for Senate trades
senate_mask = new_trades_df['chamber'] == 'Senate'

# reformat Senate dates
new_trades_df.loc[senate_mask, 'transaction_date'] = (
    pd.to_datetime(
        new_trades_df.loc[senate_mask, 'transaction_date'],
        format='%m/%d/%Y',
        errors='coerce'
    )
    .dt.strftime('%Y-%m-%d')
)

# (optional) finally convert the entire column to datetime dtype
new_trades_df['transaction_date'] = pd.to_datetime(
    new_trades_df['transaction_date'],
    errors='coerce'
)

new_trades_df['transaction_date'] = pd.to_datetime(
    new_trades_df['transaction_date'], errors='coerce'
)

# drop any rows that failed to parse
new_trades_df = new_trades_df.dropna(subset=['transaction_date'])

# filter to 2021-01-01 through 2023-12-31
start = pd.Timestamp('2014-01-01')
end   = pd.Timestamp('2023-12-31')
mask = (new_trades_df['transaction_date'] >= start) & \
       (new_trades_df['transaction_date'] <= end)

filtered = new_trades_df.loc[mask].copy()

has_ticker = (
    new_trades_df['ticker']
    .fillna('')
    .astype(str)
    .str.strip()
    .replace('', None)
    .notna()
    & (filtered['ticker'].astype(str).str.strip() != '--')
)
filtered = filtered.loc[has_ticker].copy()


[   10000    35000    75000   175000     1000   375000   750000 15000000
  3000000  1000000     8000    32500 50000000 37500000]


In [224]:
df_house_2014 = pd.read_csv(f"{PREFIX}votes/members/house/2014.csv")
df_house_2014['legislator_name'] = df_house_2014['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2015 = pd.read_csv(f"{PREFIX}votes/members/house/2015.csv")
df_house_2015['legislator_name'] = df_house_2015['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2016 = pd.read_csv(f"{PREFIX}votes/members/house/2016.csv")
df_house_2016['legislator_name'] = df_house_2016['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2017 = pd.read_csv(f"{PREFIX}votes/members/house/2017.csv")
df_house_2017['legislator_name'] = df_house_2017['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2018 = pd.read_csv(f"{PREFIX}votes/members/house/2018.csv")
df_house_2018['legislator_name'] = df_house_2018['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2019 = pd.read_csv(f"{PREFIX}votes/members/house/2019.csv")
df_house_2019['legislator_name'] = df_house_2019['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2020 = pd.read_csv(f"{PREFIX}votes/members/house/2020.csv")
df_house_2020['legislator_name'] = df_house_2020['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2021 = pd.read_csv(f"{PREFIX}votes/members/house/2021.csv")
df_house_2021['legislator_name'] = df_house_2021['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2022 = pd.read_csv(f"{PREFIX}votes/members/house/2022.csv")
df_house_2022['legislator_name'] = df_house_2022['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_house_2023 = pd.read_csv(f"{PREFIX}votes/members/house/2023.csv")
df_house_2023['legislator_name'] = df_house_2023['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2014 = pd.read_csv(f"{PREFIX}votes/members/senate/2014.csv")
df_senate_2014['legislator_name'] = df_senate_2014['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2015 = pd.read_csv(f"{PREFIX}votes/members/senate/2015.csv")
df_senate_2015['legislator_name'] = df_senate_2015['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2016 = pd.read_csv(f"{PREFIX}votes/members/senate/2016.csv")
df_senate_2016['legislator_name'] = df_senate_2016['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2017 = pd.read_csv(f"{PREFIX}votes/members/senate/2017.csv")
df_senate_2017['legislator_name'] = df_senate_2017['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2018 = pd.read_csv(f"{PREFIX}votes/members/senate/2018.csv")
df_senate_2018['legislator_name'] = df_senate_2018['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2019 = pd.read_csv(f"{PREFIX}votes/members/senate/2019.csv")
df_senate_2019['legislator_name'] = df_senate_2019['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2020 = pd.read_csv(f"{PREFIX}votes/members/senate/2020.csv")
df_senate_2020['legislator_name'] = df_senate_2020['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2021 = pd.read_csv(f"{PREFIX}votes/members/senate/2021.csv")
df_senate_2021['legislator_name'] = df_senate_2021['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2022 = pd.read_csv(f"{PREFIX}votes/members/senate/2022.csv")
df_senate_2022['legislator_name'] = df_senate_2022['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df_senate_2023 = pd.read_csv(f"{PREFIX}votes/members/senate/2023.csv")
df_senate_2023['legislator_name'] = df_senate_2023['legislator_name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()


In [225]:
# ------------------------------------------------------------
# 1) Build a deduped member-lookup table
# ------------------------------------------------------------
frames = []
for year, df_h, df_s in [
    (2014, df_house_2014, df_senate_2014),
    (2015, df_house_2015, df_senate_2015),
    (2016, df_house_2016, df_senate_2016),
    (2017, df_house_2017, df_senate_2017),
    (2018, df_house_2018, df_senate_2018),
    (2019, df_house_2019, df_senate_2019),
    (2020, df_house_2020, df_senate_2020),  
    (2021, df_house_2021, df_senate_2021),
    (2022, df_house_2022, df_senate_2022),
    (2023, df_house_2023, df_senate_2023),
]:
    # HOUSE
    house_unique = (
        df_h[['member_id','legislator_name','state']]
        .drop_duplicates()                      # <-- one row per member
        .assign(year=year, chamber='House')
    )
    house_unique['last_name'] = (
        house_unique['legislator_name']
        .str.split().str[-1]
        .str.lower()
    )
    frames.append(house_unique[['member_id','year','chamber','state','last_name']])

    # SENATE
    senate_unique = (
        df_s[['member_id','legislator_name','state']]
        .drop_duplicates()
        .assign(year=year, chamber='Senate')
    )
    senate_unique['last_name'] = (
        senate_unique['legislator_name']
        .str.split().str[-1]
        .str.lower()
    )
    frames.append(senate_unique[['member_id','year','chamber','state','last_name']])

# concatenate all years & chambers
members_df = pd.concat(frames, ignore_index=True)

# just in case there are still any dupes on the 4-key combo, keep first
members_df = members_df.drop_duplicates(
    subset=['year','chamber','state','last_name'],
    keep='first'
)

# ------------------------------------------------------------
# 2) Prepare your trades DataFrame
# ------------------------------------------------------------
# make a copy so we don’t clobber the original
trades = filtered.copy()

# extract the same merge-keys we used above
trades['year']      = trades['transaction_date'].dt.year
trades['last_name'] = trades['member'].str.split().str[-1].str.lower()
# chamber and state already exist on `filtered`

# ------------------------------------------------------------
# 3) Perform the merge
# ------------------------------------------------------------
merged = trades.merge(
    members_df,
    on=['year','chamber','state','last_name'],
    how='left'
)

# ------------------------------------------------------------
# 4) Drop any rows we still couldn’t match
# ------------------------------------------------------------
cleaned = merged.dropna(subset=['member_id'])


In [226]:


# Save the cleaned trades DataFrame
import os
os.makedirs(f"{OUTPUT_DIR}/2014-2023", exist_ok=True)
cleaned.to_csv(f"{OUTPUT_DIR}/2014-2023/stocks.csv", index=False)